# Import Libraries & Mount Google Drive 

In [2]:
import pandas as pd

# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# URA Dataset

In [104]:
ura_path = '/content/drive/My Drive/BT4222/Data/ura_5y_trans.csv' 
ura_5y_trans = pd.read_csv(ura_path)
ura_5y_trans.drop(ura_5y_trans.columns[0], axis=1, inplace=True)
ura_5y_trans['year'] = '20' + ura_5y_trans['year'].astype(str) # add '20' in front of [year] to get like 2020,2019,2018,etc.
print(ura_5y_trans.shape)
ura_5y_trans.head()

(73174, 19)


,street,project,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,x,y,nettPrice,latitude,longitude,year
0,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,524.3,-,1,220,3,5500000,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,2020
1,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,308.0,-,1,918,3,5000000,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,2018
2,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314.0,-,1,618,3,4750000,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,2018
3,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR,159.3,-,1,320,3,2630000,Terrace,5,Land,Freehold,NaN,NaN,NaN,1.292047,103.768591,2020
4,COVE DRIVE,TURQUOISE,CCR,226.0,01-05,1,920,3,3400000,Condominium,4,Strata,99 yrs lease commencing from 2007,28382.47067,25008.33592,NaN,1.244208,103.827487,2020


# Ameneties Dataset

In [16]:
amenities_path = '/content/drive/My Drive/BT4222/Data/ameneties_per_district.csv' 
ameneties_per_district = pd.read_csv(amenities_path)
ameneties_per_district.drop(ameneties_per_district.columns[0], axis=1, inplace=True)
print(ameneties_per_district.shape)
ameneties_per_district.head()

(28, 4)


,district,school,hawkercentre,supermarkets
0,1,"['NORTHLIGHT SCHOOL', 'DAMAI SECONDARY SCHOOL'...",[],[]
1,2,"['QUEENSTOWN SECONDARY SCHOOL', 'GAN ENG SENG ...",[],[]
2,3,"['SCHOOL OF SCIENCE AND TECHNOLOGY, SINGAPORE'...",['Bukit Merah Lane 1 Blk 120 (Alexandra Villag...,"['YOUR GROCER PRIVATE LIMITED', 'COLD STORAGE ..."
3,4,"['QUEENSTOWN SECONDARY SCHOOL', ""ST. MARGARET'...",['Telok Blangah Drive Blk 79 (Telok Blangah Fo...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI..."
4,5,"['JURONG SECONDARY SCHOOL', 'SCHOOL OF SCIENCE...",['West Coast Drive Blk 503 (Ayer Rajah Food Ce...,"['U STARS SUPERMARKET PTE. LTD.', 'NTUC FAIRPR..."


# Combine URA and Ameneties by District

In [71]:
# combine ura_5y_trans and ameneties_per_district
df = ura_5y_trans.join(ameneties_per_district.set_index('district'), on='district')
print(df.shape)
df
df.head()

(73174, 22)


,street,project,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,x,y,nettPrice,latitude,longitude,year,school,hawkercentre,supermarkets
0,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,524.3,-,1,220,3,5500000,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,20,"['JURONG SECONDARY SCHOOL', 'SCHOOL OF SCIENCE...",['West Coast Drive Blk 503 (Ayer Rajah Food Ce...,"['U STARS SUPERMARKET PTE. LTD.', 'NTUC FAIRPR..."
1,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,308.0,-,1,918,3,5000000,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,18,"['JURONG SECONDARY SCHOOL', 'SCHOOL OF SCIENCE...",['West Coast Drive Blk 503 (Ayer Rajah Food Ce...,"['U STARS SUPERMARKET PTE. LTD.', 'NTUC FAIRPR..."
2,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314.0,-,1,618,3,4750000,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,18,"['JURONG SECONDARY SCHOOL', 'SCHOOL OF SCIENCE...",['West Coast Drive Blk 503 (Ayer Rajah Food Ce...,"['U STARS SUPERMARKET PTE. LTD.', 'NTUC FAIRPR..."
3,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR,159.3,-,1,320,3,2630000,Terrace,5,Land,Freehold,NaN,NaN,NaN,1.292047,103.768591,20,"['JURONG SECONDARY SCHOOL', 'SCHOOL OF SCIENCE...",['West Coast Drive Blk 503 (Ayer Rajah Food Ce...,"['U STARS SUPERMARKET PTE. LTD.', 'NTUC FAIRPR..."
4,COVE DRIVE,TURQUOISE,CCR,226.0,01-05,1,920,3,3400000,Condominium,4,Strata,99 yrs lease commencing from 2007,28382.47067,25008.33592,NaN,1.244208,103.827487,20,"['QUEENSTOWN SECONDARY SCHOOL', ""ST. MARGARET'...",['Telok Blangah Drive Blk 79 (Telok Blangah Fo...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI..."


# Bus Stops Dataset

In [10]:
bus_stops_path = '/content/drive/My Drive/BT4222/Data/bus_stops.csv' 
bus_stops = pd.read_csv(bus_stops_path)
bus_stops.drop(bus_stops.columns[0], axis=1, inplace=True)
print(bus_stops.shape)
bus_stops.head()

# find the number of bus stops for each road name

(5021, 5)


,BusStopCode,RoadName,Description,Latitude,Longitude
0,481,Woodlands Rd,BT PANJANG TEMP BUS PK,1.383764,103.758300
1,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
2,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225
3,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022
4,1029,Nth Bridge Rd,Cosmic Insurance Bldg,1.296673,103.854414


# Crime Locations Dataset

In [86]:
crime_locations_path = '/content/drive/My Drive/BT4222/Data/overview_years_crimes_locations_v3.csv' 
overview_years_crimes_locations_v3 = pd.read_csv(crime_locations_path)
overview_years_crimes_locations_v3.drop(overview_years_crimes_locations_v3.columns[3], axis=1, inplace=True) # dropped 'Number (Cases)' column coz only difference between this and 'Number' is that 'Number' states the 0s
print(overview_years_crimes_locations_v3.shape)
overview_years_crimes_locations_v3.head()

# sum the number of offences in eachcategory. -> crime rate *=(do bypostal code)
# den map to district usign postal
# use THIS coz got year

# CHECK FROM 2012-2018
# DO EXTRAPOLATOIN TO FIND 2018-2021

(1680, 6)


,Year,NPC,Offence,Neighbourhood,Postal,Number
0,2012,Central Police Division - Total,Robbery,[Total]Central Police Division - Total,-,34
1,2012,Central Police Division - Total,Housebreaking,[Total]Central Police Division - Total,-,32
2,2012,Central Police Division - Total,Snatch Theft,[Total]Central Police Division - Total,-,31
3,2012,Central Police Division - Total,Theft Of Motor Vehicle,[Total]Central Police Division - Total,-,47
4,2012,Central Police Division - Total,Outrage Of Modesty,[Total]Central Police Division - Total,-,193


In [101]:
# FIND OUT IN A YEAR, FOR A POSTAL CODE, WHAT IS THE NUMBER OF OFFENCES IN EACH CATEGORY 
overview_years_crimes_locations_v3 = overview_years_crimes_locations_v3.groupby(['Year','Postal','Offence']).agg({'Number':'sum'}).reset_index()

# MAP OVERVIEW_YEARS_CRIMES_LOCATIONS_V3 [POSTAL] COLUMN INTO A NEW COLUMN NAMED [DISTRICT]

# this dictionary shows the mapping from district > 1st 2 characters of postal
district_postal = { 
  '01': ['01', '02', '03', '04', '05', '06'],
  '02': ['07', '08'],
  '03': ['14', '15', '16'],
  '04': ['09', '10'],
  '05': ['11', '12', '13'], 
  '06': ['17'],
  '07': ['18', '19'],
  '08': ['20', '21'],
  '09': ['22', '23'],
  '10': ['24', '25', '26', '27'],
  '11': ['28', '29', '30'],
  '12': ['31', '32', '33'],
  '13': ['34', '35', '36', '37'],
  '14': ['38', '39', '40', '41'],
  '15': ['42', '43', '44', '45'],
  '16': ['46', '47', '48'],
  '17': ['49', '50', '81'],
  '18': ['51', '52'],
  '19': ['53', '54', '55', '82'],
  '20': ['56', '57'],
  '21': ['58', '59'],
  '22': ['60', '61', '62', '63', '64'],
  '23': ['65', '66', '67', '68'],
  '24': ['69', '70', '71'],
  '25': ['72', '73'],
  '26': ['77', '78'],
  '27': ['75', '76'],
  '28': ['79', '80']
}

# swap dictionary mapping direction
postal_district = {k: oldk for oldk, oldv in district_postal.items() for k in oldv}

# create new column to obtain the 1st 2 characters of [Postal]
overview_years_crimes_locations_v3['postal prefix'] = overview_years_crimes_locations_v3['Postal'].astype(str).str[0:2]

# map postal to district code
overview_years_crimes_locations_v3['district']= overview_years_crimes_locations_v3['postal prefix'].map(postal_district) 
overview_years_crimes_locations_v3

,Year,Postal,Offence,Number,postal prefix,district
0,2012,-,Housebreaking,545,-,NaN
1,2012,-,Outrage Of Modesty,1398,-,NaN
2,2012,-,Robbery,284,-,NaN
3,2012,-,Snatch Theft,286,-,NaN
4,2012,-,Theft Of Motor Vehicle,584,-,NaN
...,...,...,...,...,...,...
1185,2018,828837,Housebreaking,5,82,19
1186,2018,828837,Outrage Of Modesty,23,82,19
1187,2018,828837,Robbery,3,82,19
1188,2018,828837,Snatch Theft,1,82,19


In [ ]:
# EXTRAPOLATION TO GET DATA FOR 2018-2021


## Resale Flat Prices Dataset

In [63]:
resale_flat_prices_path = '/content/drive/My Drive/BT4222/Data/resale_flat_prices_based_on_registration_date_from_jan_2017_onwards.csv' 
resale_flat_prices = pd.read_csv(resale_flat_prices_path)
resale_flat_prices.drop(resale_flat_prices.columns[0], axis=1, inplace=True)
print(resale_flat_prices.shape)
resale_flat_prices.head()

# join using district

# map to ura data set by mapping town to district number
# check if got overlaps
# convert tenure to remaining lease
# match flat_type to property type
# do appending

(92235, 14)


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,street,latitude,longitude
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,ANG MO KIO AVE 10,1.367602,103.856844
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,ANG MO KIO AVE 4,1.382319,103.838706
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,ANG MO KIO AVE 5,1.376825,103.847556
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,ANG MO KIO AVE 10,1.367602,103.856844
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,ANG MO KIO AVE 5,1.376825,103.847556


# Combine URA and Resale Flat Prices Datasets

(1) District Mapping

In [64]:
# MAP RESALE_FLAT_PRICES [TOWN] COLUMN INTO A NEW COLUMN NAMED [DISTRICT]

# this dictionary shows the mapping from district > town
district_town = {
  '01': ['Raffles Place', 'Cecil', 'Marina', 'People’s Park'],
  '02': ['Anson', 'Tanjong Pagar'],
  '03': ['Queenstown', 'Tiong Bahru'],
  '04': ['Telok Blangah', 'Harbourfront'],
  '05': ['Pasir Panjang', 'Hong Leong Garden', 'Clementi New Town'], 
  '06': ['High Street', 'Beach Road'],
  '07': ['Middle Road', 'Golden Mile'],
  '08': ['Little India'],
  '09': ['Orchard', 'Cairnhill', 'River Valley'],
  '10': ['Ardmore', 'Bukit Timah', 'Holland Road', 'Tanglin'],
  '11': ['Watten Estate', 'Novena', 'Thomson'],
  '12': ['Balestier', 'Toa Payoh','Serangoon'],
  '13': ['Macpherson', 'Braddell'],
  '14': ['Geylang', 'Eunos'],
  '15': ['Katong', 'Joo Chiat', 'Amber Road'],
  '16': ['Bedok', 'Upper East Coast', 'Eastwood', 'Kew Drive'],
  '17': ['Loyang', 'Changi'],
  '18': ['Tampines', 'Pasir Ris'],
  '19': ['Serangoon Garden', 'Hougang', 'Punggol'],
  '20': ['Bishan', 'Ang Mo Kio'],
  '21': ['Upper Bukit Timah', 'Clementi Park', 'Ulu Pandan'],
  '22': ['Jurong'],
  '23': ['Hillview', 'Dairy Farm', 'Bukit Panjang', 'Choa Chu Kang'],
  '24': ['Lim Chu Kang', 'Tengah'],
  '25': ['Kranji', 'Woodgrove'],
  '26': ['Upper Thomson', 'Springleaf'],
  '27': ['Yishun', 'Sembawang'],
  '28': ['Seletar']
}
district_town = {key: [ele.upper() for ele in district_town[key] ] for key in district_town } # convert all values to upper case

# swap dictionary mapping direction
town_district = {k: oldk for oldk, oldv in district_town.items() for k in oldv}

# map town to district code
resale_flat_prices['town'].str.upper()  # convert and ensure all town string in the data are in upper case 
resale_flat_prices['district']= resale_flat_prices['town'].map(town_district)

(2) Tenure conversion to Remaining Lease

In [73]:
resale_flat_prices

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,street,latitude,longitude,district
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,ANG MO KIO AVE 10,1.367602,103.856844,20
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,ANG MO KIO AVE 4,1.382319,103.838706,20
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,ANG MO KIO AVE 5,1.376825,103.847556,20
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,ANG MO KIO AVE 10,1.367602,103.856844,20
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,ANG MO KIO AVE 5,1.376825,103.847556,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92230,2021-02,YISHUN,EXECUTIVE,406,YISHUN AVE 6,10 TO 12,148.0,Maisonette,1988,66 years 08 months,655000.0,YISHUN AVE 6,1.426142,103.849653,27
92231,2021-02,YISHUN,EXECUTIVE,355,YISHUN RING RD,10 TO 12,146.0,Maisonette,1988,66 years 10 months,650000.0,YISHUN RING ROAD,1.424340,103.845070,27
92232,2021-02,YISHUN,EXECUTIVE,364,YISHUN RING RD,01 TO 03,146.0,Maisonette,1988,66 years 02 months,607000.0,YISHUN RING ROAD,1.424340,103.845070,27
92233,2021-02,YISHUN,EXECUTIVE,828,YISHUN ST 81,01 TO 03,142.0,Apartment,1988,66 years,670000.0,YISHUN ST 81,1.415253,103.837021,27


In [72]:
df

,street,project,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,x,y,nettPrice,latitude,longitude,year,school,hawkercentre,supermarkets
0,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,524.3,-,1,220,3,5500000,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,20,"['JURONG SECONDARY SCHOOL', 'SCHOOL OF SCIENCE...",['West Coast Drive Blk 503 (Ayer Rajah Food Ce...,"['U STARS SUPERMARKET PTE. LTD.', 'NTUC FAIRPR..."
1,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,308.0,-,1,918,3,5000000,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,18,"['JURONG SECONDARY SCHOOL', 'SCHOOL OF SCIENCE...",['West Coast Drive Blk 503 (Ayer Rajah Food Ce...,"['U STARS SUPERMARKET PTE. LTD.', 'NTUC FAIRPR..."
2,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314.0,-,1,618,3,4750000,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,18,"['JURONG SECONDARY SCHOOL', 'SCHOOL OF SCIENCE...",['West Coast Drive Blk 503 (Ayer Rajah Food Ce...,"['U STARS SUPERMARKET PTE. LTD.', 'NTUC FAIRPR..."
3,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR,159.3,-,1,320,3,2630000,Terrace,5,Land,Freehold,NaN,NaN,NaN,1.292047,103.768591,20,"['JURONG SECONDARY SCHOOL', 'SCHOOL OF SCIENCE...",['West Coast Drive Blk 503 (Ayer Rajah Food Ce...,"['U STARS SUPERMARKET PTE. LTD.', 'NTUC FAIRPR..."
4,COVE DRIVE,TURQUOISE,CCR,226.0,01-05,1,920,3,3400000,Condominium,4,Strata,99 yrs lease commencing from 2007,28382.47067,25008.33592,NaN,1.244208,103.827487,20,"['QUEENSTOWN SECONDARY SCHOOL', ""ST. MARGARET'...",['Telok Blangah Drive Blk 79 (Telok Blangah Fo...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73169,YISHUN CENTRAL 1,NORTH PARK RESIDENCES,OCR,51.0,06-10,1,1020,3,855000,Apartment,27,Strata,99 yrs lease commencing from 2015,28390.14699,45560.43713,NaN,1.424791,103.836792,20,"['CHUNG CHENG HIGH SCHOOL (YISHUN)', 'YISHUN S...","['Yishun Park Hawker Centre', 'Yishun Ring Roa...","['HAO MART PTE. LTD.', 'SHENG SIONG SUPERMARKE..."
73170,YISHUN CENTRAL 1,NORTH PARK RESIDENCES,OCR,70.0,11-15,1,920,3,1070000,Apartment,27,Strata,99 yrs lease commencing from 2015,28390.14699,45560.43713,NaN,1.424791,103.836792,20,"['CHUNG CHENG HIGH SCHOOL (YISHUN)', 'YISHUN S...","['Yishun Park Hawker Centre', 'Yishun Ring Roa...","['HAO MART PTE. LTD.', 'SHENG SIONG SUPERMARKE..."
73171,YISHUN CENTRAL 1,NORTH PARK RESIDENCES,OCR,77.0,06-10,1,720,2,1290000,Apartment,27,Strata,99 yrs lease commencing from 2015,28390.14699,45560.43713,NaN,1.424791,103.836792,20,"['CHUNG CHENG HIGH SCHOOL (YISHUN)', 'YISHUN S...","['Yishun Park Hawker Centre', 'Yishun Ring Roa...","['HAO MART PTE. LTD.', 'SHENG SIONG SUPERMARKE..."
73172,YISHUN CENTRAL 1,NORTH PARK RESIDENCES,OCR,77.0,06-10,1,121,2,1330000,Apartment,27,Strata,99 yrs lease commencing from 2015,28390.14699,45560.43713,NaN,1.424791,103.836792,21,"['CHUNG CHENG HIGH SCHOOL (YISHUN)', 'YISHUN S...","['Yishun Park Hawker Centre', 'Yishun Ring Roa...","['HAO MART PTE. LTD.', 'SHENG SIONG SUPERMARKE..."
